In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Montando o Google Drive no ambiente do Google Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Definindo os caminhos dos arquivos CSV de exportação e importação
file_path1 = '/content/drive/MyDrive/projeto_dnc/EXP_2024.csv'
file_path3 = '/content/drive/MyDrive/projeto_dnc/IMP_2024.csv.crdownload'

In [ ]:
df1 = pd.read_csv(file_path1, sep=';')
df3 = pd.read_csv(file_path3, sep=';')

In [ ]:
# Removendo valores ausentes e duplicados
df1.dropna(inplace=True)
df1.drop_duplicates(inplace=True)
df3.dropna(inplace=True)
df3.drop_duplicates(inplace=True)

In [ ]:
df1.head()

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
0,2024,4,94036000,11,245,SP,4,817600,8,425,20423
1,2024,11,87088000,11,493,SP,1,817800,418615,60355,470512
2,2024,3,2071400,10,173,SP,1,817800,94080,94080,237562
3,2024,4,90303390,11,158,MG,1,817800,10,12,1704
4,2024,8,40129090,10,158,SC,4,817700,26,26,950


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1603796 entries, 0 to 1603795
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   CO_ANO      1603796 non-null  int64 
 1   CO_MES      1603796 non-null  int64 
 2   CO_NCM      1603796 non-null  int64 
 3   CO_UNID     1603796 non-null  int64 
 4   CO_PAIS     1603796 non-null  int64 
 5   SG_UF_NCM   1603796 non-null  object
 6   CO_VIA      1603796 non-null  int64 
 7   CO_URF      1603796 non-null  int64 
 8   QT_ESTAT    1603796 non-null  int64 
 9   KG_LIQUIDO  1603796 non-null  int64 
 10  VL_FOB      1603796 non-null  int64 
dtypes: int64(10), object(1)
memory usage: 134.6+ MB


In [ ]:
df3.head()

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO
0,2024,10,84818092,11,161,SC,1,927800,344.0,90.0,6823.0,77.0,10.0
1,2024,4,90304090,11,249,SP,4,817700,1.0,9.0,4890.0,147.0,7.0
2,2024,1,39172300,10,23,PE,4,417901,1128.0,1128.0,28129.0,20688.0,7.0
3,2024,6,85444200,10,160,MG,4,817700,824.0,824.0,66958.0,11276.0,45.0
4,2024,10,84439942,10,190,SC,4,817600,3.0,3.0,2060.0,37.0,0.0


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183522 entries, 0 to 183521
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   CO_ANO      183522 non-null  int64  
 1   CO_MES      183522 non-null  int64  
 2   CO_NCM      183522 non-null  int64  
 3   CO_UNID     183522 non-null  int64  
 4   CO_PAIS     183522 non-null  int64  
 5   SG_UF_NCM   183522 non-null  object 
 6   CO_VIA      183522 non-null  int64  
 7   CO_URF      183522 non-null  int64  
 8   QT_ESTAT    183522 non-null  float64
 9   KG_LIQUIDO  183522 non-null  float64
 10  VL_FOB      183522 non-null  float64
 11  VL_FRETE    183522 non-null  float64
 12  VL_SEGURO   183522 non-null  float64
dtypes: float64(5), int64(7), object(1)
memory usage: 19.6+ MB


In [ ]:
# Selecionando apenas as colunas relevantes para análise de exportação e importação
dfexp = df1[['CO_NCM', 'VL_FOB']]
dfimp = df3[['CO_NCM', 'VL_FOB']]

In [ ]:
# Agrupando os dados de importação por NCM
dfimp = dfimp.groupby(dfimp.CO_NCM).sum().reset_index()

In [ ]:
dfimp

,CO_NCM,VL_FOB
0,1012100,255050.0
1,1012900,1500.0
2,1022190,95000.0
3,1061900,18162.0
4,2013000,10786727.0
...,...,...
7164,97019900,88520.0
7165,97029000,7482.0
7166,97039000,326672.0
7167,97051000,40290.0


In [ ]:
# Agrupando os dados de exportação por NCM somando os valores FOB
dfexp = dfexp.groupby(dfexp.CO_NCM).sum().reset_index()

In [ ]:
dfexp

,CO_NCM,VL_FOB
0,1012100,3134431
1,1012900,5453342
2,1013000,311
3,1019000,1406
4,1022110,16413
...,...,...
8014,97040000,195716
8015,97051000,1962
8016,97052900,50
8017,97061000,861225


In [ ]:
df_ncm = pd.read_csv('/content/drive/MyDrive/projeto_dnc/NCM.csv', sep=';', encoding='latin1')

In [ ]:
df_ncm.head()

,CO_NCM,CO_UNID,CO_SH6,CO_PPE,CO_PPI,CO_FAT_AGREG,CO_CUCI_ITEM,CO_CGCE_N3,CO_SIIT,CO_ISIC_CLASSE,CO_EXP_SUBSET,NO_NCM_POR,NO_NCM_ESP,NO_NCM_ING
0,59112090,10,591120,3990.0,3990.0,3.0,65773,270,4000.0,1399,1602.0,"Gazes e telas para peneirar, mesmo confecciona...","Gasas y telas p/cener, de otras materias textiles","Gauzes and screens to drizzle, of oth.textile ..."
1,59113100,10,591131,3838.0,3705.0,3.0,65773,270,4000.0,1399,1602.0,"Tecidos e feltros, sem fim ou com dispositivos...","Tela y fieltro, util.en máqs.p/fabr.de papel, ...",Fabric and felt used in mach.f/prod.of paper w...
2,59113200,10,591132,3838.0,3705.0,3.0,65773,270,4000.0,1399,1602.0,"Tecidos e feltros, sem fim ou com dispositivos...","Tela y fieltro, util.en máqs.p/fabr.de papel, ...",Fabric and felt used in mach.f/prod.of paper w...
3,59114000,10,591140,3990.0,3990.0,3.0,65773,270,4000.0,1399,1602.0,"Tecidos filtrantes e tecidos espessos, compree...","Capachos y telas gruesas, util.prensas de acei...","Fabrics of filter/thick, used in oil presses, ..."
4,51121100,10,511211,3990.0,3990.0,3.0,65422,240,4000.0,1312,1699.0,Tecidos de lã penteada ou de pelos finos pente...,"Tejido de lana/pelos finos, pein.(cont>=85%), ...","Combed fabr.of wool/fine anim.hair(con>=85%), ..."


In [ ]:
# Selecionando apenas as colunas relevantes da tabela NCM
df_ncm = df_ncm[['CO_NCM', 'NO_NCM_POR']]

In [ ]:
df_ncm.head()

,CO_NCM,NO_NCM_POR
0,59112090,"Gazes e telas para peneirar, mesmo confecciona..."
1,59113100,"Tecidos e feltros, sem fim ou com dispositivos..."
2,59113200,"Tecidos e feltros, sem fim ou com dispositivos..."
3,59114000,"Tecidos filtrantes e tecidos espessos, compree..."
4,51121100,Tecidos de lã penteada ou de pelos finos pente...


In [ ]:
# Realizando a junção entre dfexp e df_ncm
dfexp = dfexp.merge(df_ncm, on='CO_NCM', how='inner')

In [ ]:
dfexp

,CO_NCM,VL_FOB,NO_NCM_POR
0,1012100,3134431,Cavalos reprodutores de raça pura
1,1012900,5453342,"Cavalos, exceto reprodutores de raça pura"
2,1013000,311,"Animais espécie asinina/muar, reprodutores de ..."
3,1019000,1406,"Animais espécie asinina/muar, exceto reproduto..."
4,1022110,16413,"Bovinos reprodutores raça pura, prenhe ou com ..."
...,...,...,...
8014,97040000,195716,"Selos postais, selos fiscais, marcas postais, ..."
8015,97051000,1962,Coleções e peças de coleção que apresentem um ...
8016,97052900,50,Outras coleções e peças de coleção que apresen...
8017,97061000,861225,Antiguidades com mais de 250 anos


In [ ]:
# Realizando a junção entre dfimp e df_ncm
dfimp = dfimp.merge(df_ncm, on='CO_NCM', how='inner')

In [ ]:
dfimp

,CO_NCM,VL_FOB,NO_NCM_POR
0,1012100,255050.0,Cavalos reprodutores de raça pura
1,1012900,1500.0,"Cavalos, exceto reprodutores de raça pura"
2,1022190,95000.0,Outros bovinos reprodutores de raça pura
3,1061900,18162.0,Outros mamíferos vivos
4,2013000,10786727.0,"Carnes desossadas de bovino, frescas ou refrig..."
...,...,...,...
7164,97019900,88520.0,"Outros quadros, pinturas e desenhos, feitos in..."
7165,97029000,7482.0,"Outras gravuras, estampas e litografias, origi..."
7166,97039000,326672.0,Outras produções originais de arte estatuária ...
7167,97051000,40290.0,Coleções e peças de coleção que apresentem um ...


In [ ]:
# Calculando o valor total geral
total_exp = dfexp['VL_FOB'].sum()

# Criando uma nova coluna com a participação percentual de cada produto
dfexp['PERC_PARTICIPACAO_EXP'] = ((dfexp['VL_FOB'] / total_exp) * 100).round(2)

# Ordenando do maior para o menor
dfexp = dfexp.sort_values(by='PERC_PARTICIPACAO_EXP', ascending=False)

In [ ]:
dfexp.head()

,CO_NCM,VL_FOB,NO_NCM_POR,PERC_PARTICIPACAO_EXP
1271,27090010,44963878520,Óleos brutos de petróleo,13.34
697,12019000,42941894470,"Soja, mesmo triturada, exceto para semeadura",12.74
1221,26011100,26574698749,"Minérios de ferro e seus concentrados, exceto ...",7.88
890,17011400,15925320657,Outros açúcares de cana,4.72
590,9011110,11331933183,"Café não torrado, não descafeinado, em grão",3.36


In [ ]:
# Calculando o valor total geral
total_imp = dfimp['VL_FOB'].sum()

# Criando uma nova coluna com a participação percentual de cada produto
dfimp['PERC_PARTICIPACAO_IMP'] = ((dfimp['VL_FOB'] / total_imp) * 100).round(2)

# Ordenando do maior para o menor
dfimp = dfimp.sort_values(by='PERC_PARTICIPACAO_IMP', ascending=False)

In [ ]:
dfimp

,CO_NCM,VL_FOB,NO_NCM_POR,PERC_PARTICIPACAO_IMP
649,27090010,667621337.0,Óleos brutos de petróleo,3.00
4963,84111200,662603315.0,Turborreatores de empuxo superior a 25 kN,2.98
4967,84119100,631171645.0,Partes de turborreatores ou de turbopropulsores,2.84
6449,87036000,607121371.0,"Outros veículos, equipados para propulsão, sim...",2.73
662,27101921,583996553.0,Gasóleo (óleo diesel),2.62
...,...,...,...,...
1,1012900,1500.0,"Cavalos, exceto reprodutores de raça pura",0.00
7168,97053900,197818.0,Outras coleções e peças de coleção que aprese...,0.00
7154,96161000,142002.0,"Vaporizadores de toucador, suas armações e cab...",0.00
7155,96162000,404868.0,Borlas ou esponjas para pós ou para aplicação ...,0.00


In [ ]:
# Renomeando a coluna VL_FOB
dfexp.rename(columns={'VL_FOB': 'VL_FOB_EXP'}, inplace=True)
dfimp.rename(columns={'VL_FOB': 'VL_FOB_IMP'}, inplace=True)

# Total de exportação para calcular participação
total_exp = dfexp['VL_FOB_EXP'].sum()
dfexp['PERC_PARTICIPACAO_EXP'] = (dfexp['VL_FOB_EXP'] / total_exp * 100).round(2)

total_imp = dfimp['VL_FOB_IMP'].sum()
dfimp['PERC_PARTICIPACAO_IMP'] = (dfimp['VL_FOB_IMP'] / total_imp * 100).round(2)

# Unindo os dois DataFrames pelas colunas CO_NCM e NO_NCM_POR
df_final = pd.merge(dfexp, dfimp, on=['CO_NCM', 'NO_NCM_POR'], how='outer')

# Preenche valores ausentes com 0
df_final[['VL_FOB_EXP', 'PERC_PARTICIPACAO_EXP', 'VL_FOB_IMP', 'PERC_PARTICIPACAO_IMP']] = \
    df_final[['VL_FOB_EXP', 'PERC_PARTICIPACAO_EXP', 'VL_FOB_IMP', 'PERC_PARTICIPACAO_IMP']].fillna(0)

# Calcula diferença entre exportações e importações
df_final['DIF_EXP_IMP'] = df_final['VL_FOB_EXP'] - df_final['VL_FOB_IMP']

# Reorganizando as colunas
df_final = df_final[['CO_NCM', 'NO_NCM_POR', 'VL_FOB_EXP', 'VL_FOB_IMP',
                     'PERC_PARTICIPACAO_EXP', 'PERC_PARTICIPACAO_IMP', 'DIF_EXP_IMP']]
df_final = df_final.sort_values(by='DIF_EXP_IMP', ascending=False)

In [ ]:
df_final

,CO_NCM,NO_NCM_POR,VL_FOB_EXP,VL_FOB_IMP,PERC_PARTICIPACAO_EXP,PERC_PARTICIPACAO_IMP,DIF_EXP_IMP
1300,27090010,Óleos brutos de petróleo,4.496388e+10,667621337.0,13.34,3.00,4.429626e+10
705,12019000,"Soja, mesmo triturada, exceto para semeadura",4.294189e+10,8160200.0,12.74,0.04,4.293373e+10
1245,26011100,"Minérios de ferro e seus concentrados, exceto ...",2.657470e+10,339184.0,7.88,0.00,2.657436e+10
907,17011400,Outros açúcares de cana,1.592532e+10,311087.0,4.72,0.00,1.592501e+10
595,9011110,"Café não torrado, não descafeinado, em grão",1.133193e+10,0.0,3.36,0.00,1.133193e+10
...,...,...,...,...,...,...,...
2897,31021010,"Ureia, mesmo em solução aquosa, com teor de ni...",6.207602e+06,263903643.0,0.00,1.19,-2.576960e+08
6343,84119100,Partes de turborreatores ou de turbopropulsores,3.725421e+08,631171645.0,0.11,2.84,-2.586296e+08
5907,74031100,"Cátodos e seus elementos de cobre refinado, em...",3.400000e+01,297856923.0,0.00,1.34,-2.978569e+08
6338,84111200,Turborreatores de empuxo superior a 25 kN,1.109142e+08,662603315.0,0.03,2.98,-5.516891e+08


In [ ]:
df_final.describe()

,CO_NCM,VL_FOB_EXP,PERC_PARTICIPACAO_EXP,VL_FOB_IMP,PERC_PARTICIPACAO_IMP,DIF_EXP_IMP
count,8.774000e+03,8.774000e+03,8774.000000,8.774000e+03,8774.000000,8.774000e+03
mean,5.300153e+07,3.841420e+07,0.011116,2.537415e+06,0.010955,3.587678e+07
std,2.797256e+07,7.912945e+08,0.234743,1.872368e+07,0.084232,7.868581e+08
min,1.012100e+06,0.000000e+00,0.000000,0.000000e+00,0.000000,-6.071214e+08
25%,2.924192e+07,7.725250e+03,0.000000,2.552500e+03,0.000000,-8.922725e+04
50%,5.208260e+07,1.953385e+05,0.000000,1.236305e+05,0.000000,9.116000e+03
75%,8.426970e+07,2.357003e+06,0.000000,8.763438e+05,0.000000,1.210475e+06
max,9.706900e+07,4.496388e+10,13.340000,6.676213e+08,3.000000,4.429626e+10
